In [ ]:
def date_range_check(
    start_date_str, end_date_str, start_time_collection, end_time_collection
):
    # Convert input strings to datetime objects
    start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
    end_date = datetime.strptime(end_date_str, "%Y-%m-%d")

    # Convert list of date strings to datetime objects
    x = datetime.strptime(start_time_collection, "%Y-%m-%d")
    y = datetime.strptime(end_time_collection, "%Y-%m-%d")

    # Combined list of dates
    all_dates = [x, y]

    # Check 1: start date must be older than end date
    if start_date >= end_date:
        return "Fail: Start date should be older than end date"

    # Check 2: end date must be equal to or more recent than any date on the first or second list
    if not any(end_date >= d for d in all_dates):
        return (
            "Fail: End date is not equal to or more recent than any date in the lists"
        )

    # Check 3: start date must be older than any date on the first or second list
    if not any(start_date < d for d in all_dates):
        return "Fail: Start date is not older than any date in the lists"

    return "Pass"

In [ ]:
def aoi_filterbounds_check(collection, aoi):
    aoi_test = (
            ee.ImageCollection(collection)
            .filterBounds(aoi)
            .limit(2)
            .size()
            .getInfo()
        )
    if aoi_test > 0:
        return "Pass"
    else:
        return "Fail: No alerts found on AOI"

In [ ]:
import ee
from sepal_ui.scripts.utils import init_ee
init_ee()

ccdc_layer = 'projects/ee-joseserafini-fao/assets/Change_alerts_2022-11-09_18-45-47'

def get_alert_dates_GLAD_L():
    """Retrieve start and end dates for GLAD L collection."""
    start_date = "2018-01-01"
    end_date = ee.Date(ee.ImageCollection("projects/glad/alert/UpdResult")
                       .sort("system:time_start", False).first()
                       .get("system:time_start")).format("yyyy-MM-dd").getInfo()
    return [start_date, end_date]

def get_alert_dates_ccdc(alert_collection):
    """Retrieve start and end dates for an CCDC alert collection."""
    start_date = ee.Date(ee.Image(alert_collection).get("system:time_start")).format("yyyy-MM-dd").getInfo()
    end_date = ee.Date(ee.Image(alert_collection).get("system:time_end")).format("yyyy-MM-dd").getInfo()
    return [start_date, end_date]

def get_alert_dates_GLAD_S2():
    """Retrieve start and end dates for GLAD S2 collection."""
    start_date = "2019-01-01"
    end_date = (ee.Image("projects/glad/S2alert/alert").get("date").getInfo())
    return [start_date, end_date]

def get_alert_dates_RADD():
    """Retrieve start and end dates for RADD collection."""
    radd = ee.ImageCollection("projects/radar-wur/raddalert/v1").filterMetadata("layer", "contains", "alert")    
    start_date = "2019-01-01"
    end_date = (radd.sort("system:time_end", False).first().get("version_date").getInfo())
    return [start_date, end_date]

dates_radd = get_alert_dates_RADD()
dates_gladL = get_alert_dates_GLAD_L()
dates_gladS2 = get_alert_dates_GLAD_S2()

# Define alert metadata in a dictionary
alert_metadata = {
    "RADD": {
        "collection":"projects/radar-wur/raddalert/v1",
        "start_date": dates_radd[0],
        "end_date": dates_radd[1]
    },
    "GLADS2": {
        "collection": "projects/glad/S2alert",
        "start_date": dates_gladS2[0],
        "end_date": dates_gladS2[1]
    },
    "GLADL": {
        "collection": "projects/glad/alert/2018final",
        "start_date": dates_gladL[0],
        "end_date": dates_gladL[1]
    }
}
if ccdc_layer:
    dates_ccdc = get_alert_dates_ccdc(ccdc_layer)
    alert_metadata["CCDC"] = {
        "collection":ccdc_layer,
        "start_date": dates_ccdc[0],
        "end_date": dates_ccdc[1]}

alert_metadata

In [ ]:
def create_alerts_dictionary(ccdc_layer):
    dates_radd = get_alert_dates_RADD()
    dates_gladL = get_alert_dates_GLAD_L()
    dates_gladS2 = get_alert_dates_GLAD_S2()
    
    # Define alert metadata in a dictionary
    alert_metadata = {
        "RADD": {
            "collection":"projects/radar-wur/raddalert/v1",
            "start_date": dates_radd[0],
            "end_date": dates_radd[1]
        },
        "GLADS2": {
            "collection": "projects/glad/S2alert",
            "start_date": dates_gladS2[0],
            "end_date": dates_gladS2[1]
        },
        "GLADL": {
            "collection": "projects/glad/alert/2018final",
            "start_date": dates_gladL[0],
            "end_date": dates_gladL[1]
        }
    }
    if ccdc_layer:
        dates_ccdc = get_alert_dates_ccdc(ccdc_layer)
        alert_metadata["CCDC"] = {
            "collection":ccdc_layer,
            "start_date": dates_ccdc[0],
            "end_date": dates_ccdc[1]}

    return alert_metadata

In [ ]:
def create_available_alert_dictionary(dataset, aoi, date1, date2):
    #Perform test over alert collection to check date and aoi availability
    result = []
    for name, data in dataset.items():
        start_date = data['start_date']
        end_date = data['end_date']
        if name == 'CCDC':
            collection = ee.ImageCollection.fromImages([data['collection']])
        else:
            collection = data['collection']

        # Apply the functions
        check1 = date_range_check(date1, date2, start_date, start_date)
        check2 = aoi_filterbounds_check(collection, aoi)
        if check1 == 'Pass' and check2 == 'Pass':
            result.append(name)
        else
            result

In [ ]:
import ee

# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize()

# Load the collection
collection = ee.ImageCollection("projects/glad/alert/UpdResult")

# Get the first image from the collection
first_image = collection.first()

# Get the list of band names
bands = first_image.bandNames().getInfo()

# Filter bands that start with 'alertDate'
alert_date_bands = [band for band in bands if band.startswith('alertDate')]

if len(alert_date_bands) != 2:
    raise ValueError("Expected exactly two bands starting with 'alertDate'")

# Step 1: Remove "alertDate" from each element
modified_list = [element.replace('alertDate', '') for element in alert_date_bands]

# Step 2 and 3: Extract numeric part and convert to integer
numeric_values = [int(value) for value in modified_list]

# Step 4: Find the lowest digit
lowest_digit = 2000 + min(numeric_values)

print(lowest_digit)
